# Crowd dynamics simulation — a multi-agent system based on CA

# Tasks

## Neighbourhood
Fill in the missing part of the *\_\_init\_\_* method in the *Board* class. Prepare two versions: one with Moore and one with von Neumann neighbourhood. Do not initialise neighbourhoods for border cells.
## Static potential field (2 points)

### Implement method *calculateField* in *Board* class:
* Create a list of points (*toCheck*) for which the static field should be recalculated (at the beginning all *staticFields* are equal to 100000). 
    * For all exits (points with *pointType == 2*) change the *staticField* to 0. 
    * Add all exits' neighbours to the *toCheck* list.
* Until the list *toCheck* is empty:
    * chech whether the first element of the list changed its *staticField* (call *calcStaticField*).
    * If so, add all its neighbours to the *toCheck* list.
    * Remove the first element from the list.

### Implement method *calcStaticField* in *Point* class:
* Find the smallest *staticField* in the cell's neighbourhood.
* If the cell's *staticField* is greater than the found value + 1, change the *staticField* to the found value + 1 and return *True*, otherwise do not change anything and return *False*.

### Implement *randomBoard* method:
* Add some walls and pedestrians, add at least one exit.
* Try to do this randomly.

## Naive Implementation ( 2 points)

### Implement *move* method:
* If the cell represents a pedestrian (*pointType == 3*), move the pedestrian to the neighbouring cell with the lowest *staticField*.

Run the simulation and observe what is happening. What's wrong? Try to find why some artefacts can be observed.

## Improvements (2 points)
Two main reasons responsible for the errors are:
* No "exit" mechanism. After reaching the exit, the agent should be removed.
* No cells synchronisation. One should note that agent moving down and right can reach the destination in one iteration.
* To fix this issue add a boolean value *isBlocked = false* to the *Point* class. After the agent moves, the *isBlocked* value of the occupied cell should change to *True*. Remember to "unblock" all cells at the beginning of each iteration. 

Correct the errors and create a complex board with walls, many pedestrians and at least two exits.




In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import itertools
from IPython.display import HTML

In [19]:
class Point:
    def __init__(self):
        self.neighbours = []
    # 0 - floor, 1 - wall, 2 - exit, 3 - pedestrian
        self.pointType = 0
        self.staticField = 100000
        self.isBlocked = False

    def clear(self):
        self.staticField = 100000


    def calcStaticField(self):
        minVal = 100000
        for i in self.neighbours:
            if i.staticField < minVal:
                minVal = i.staticField
        if(self.staticField > minVal+1):
            self.staticField = minVal+1
            return True
        return False

    def move(self):
        # TODO
        if(self.pointType == 3 and not self.isBlocked):
            lowestSFNeighbour = self
            for i in self.neighbours:
                if(i.staticField<lowestSFNeighbour.staticField and i.pointType not in [1,3]):
                    lowestSFNeighbour = i
            self.pointType=0
            self.isBlocked = True
            if(lowestSFNeighbour.pointType != 2):
                lowestSFNeighbour.pointType = 3
            lowestSFNeighbour.isBlocked = True

        return


    def getColour(self):
        if self.pointType == 0:
            # Floor
            # Return visualization of staticField
            return self.staticField*3
            return 0
        elif self.pointType == 1:
            # Wall
            return 255
        elif self.pointType == 2:
            # Exit
            return 64
        else:
            # Pedestrian
            return 128
        


In [41]:
class Board:
    
    def __init__(self, xSize, ySize):
        self.xSize = xSize
        self.ySize = ySize
        self.points = []
        for i in range(0, xSize):
            row = []
            for j in range(0, ySize):
                row.append(Point())
            self.points.append(row)
                
        for i in range(1, xSize-1):
            for j in range(1, ySize-1):
                # TODO initialize neighbourhood here
                # Moore
                tmpList = [self.points[i-1][j-1], self.points[i-1][j], self.points[i-1][j+1],
                          self.points[i][j-1], self.points[i][j+1], self.points[i+1][j-1], self.points[i+1][j], self.points[i+1][j+1]]
    
                self.points[i][j].neighbours = tmpList
        
                #Von Neumann
                vnList = [self.points[i-1][j], self.points[i+1][j], self.points[i][j+1], self.points[i][j-1]]
                self.points[i][j].neighbours = vnList
                
                                                    
    def randomBoard(self, no_of_horizontal=1, no_of_vertical=1, no_of_objects=3, no_of_exits=1):
        # walls horizontal 
        for i in range(no_of_horizontal):        
            x = random.randrange(1, (self.xSize-1)//2)
            xend = random.randrange(x, self.xSize-1)
            y = random.randrange(1, (self.ySize-1)//2)
            for j in range(x,xend):
                self.points[j][y].pointType = 1

        # vertical
        for i in range(no_of_vertical):        
            x = random.randrange(1, (self.xSize-1)//2)
            y = random.randrange(1, (self.ySize-1)//2)
            yend = random.randrange(y, self.ySize-1)
            for j in range(y,yend):
                self.points[x][j].pointType = 1

        # moving obj
        for i in range(no_of_objects):
            x = random.randrange(1, self.xSize-1)
            y = random.randrange(1, self.ySize-1)
            self.points[x][y].pointType = 3

        #exits
        for i in range(no_of_exits):
            x = random.randrange(1, self.xSize-1)
            y = random.randrange(1, self.ySize-1)
            self.points[x][y].pointType = 2

        self.calculateField()
        return

        
    def iteration(self):
        for i in range(1,len(self.points)-1):
            for j in range(1, len(self.points[0])-1):
                self.points[i][j].isBlocked = False
        for i in range(1,len(self.points)-1):
            for j in range(1, len(self.points[0])-1):
                self.points[i][j].move()

    
    def clear(self):
        for i in range(1,len(self.points)-1):
            for j in range(1, len(self.points[0])-1):
                self.points[i][j].clear()
        self.calculateField()
    
    def redraw(self):
        # Returns numpy array of colours
        fig = np.zeros((len(self.points),len(self.points[0])))
        for i in range(0,len(self.points)):
            for j in range(0, len(self.points[0])):
                fig[i][j] = self.points[i][j].getColour()
        return fig
    
    def getFieldMap(self):
        # Returns numpy array of fieldValues
        fig = np.zeros((len(self.points),len(self.points[0])))
        for i in range(0,len(self.points)):
            for j in range(0, len(self.points[0])):
                fig[i][j] = self.points[i][j].staticField
        return fig
    
    def calculateField(self):
        toCheck = []
        for i in range(1, self.xSize-1):
            for j in range(1, self.ySize-1):
                if(self.points[i][j].pointType == 2):
                    self.points[i][j].staticField = 0
                    toCheck.extend(self.points[i][j].neighbours)
        while(toCheck):
            didChanged = toCheck[0].calcStaticField()
            if(didChanged):
                toCheck.extend(toCheck[0].neighbours)
            toCheck.pop(0)
        return
                
            

In [42]:
def update(frameNum, board, im):
    if(frameNum == 0):
        im.set_data(board.redraw()) 
    else:   
        board.iteration()
        im.set_data(board.redraw()) 
    return im,

In [43]:
%%capture
# Parameters
steps = 100
# Config
fig, ax = plt.subplots(figsize=(10,10))
plt.axis('off')


In [50]:
#Run Simulation
import random 

board = Board(30,30)

board.clear()
board.randomBoard(3,3,5,2)


In [51]:

im = ax.imshow(board.redraw(),  vmin=0, vmax=255)
ani = animation.FuncAnimation(fig, update, frames = steps, fargs = [board, im])
HTML(ani.to_jshtml())